# Router Engine

![Router Engine Diagram](assets/router-engine.png)]

In [ ]:
from helper import get_openai_api_key

OPENAI_API_KEY = get_openai_api_key()

In [ ]:
import nest_asyncio

nest_asyncio.apply()

### Load Data

In [ ]:
from llama_index.core import SimpleDirectoryReader

In [ ]:
# Load documents
documents = SimpleDirectoryReader(input_files=['assets/metagpt.pdf']).load_data()

### Define LLM and Embedding model

In [ ]:
from llama_index.core.node_parser import SentenceSplitter

splitter = SentenceSplitter(chunk_size=1024)
nodes = splitter.get_nodes_from_documents(documents)

In [ ]:
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

Settings.llm = OpenAI(model='gpt-3.5-turbo')
Settings.embed_model = OpenAIEmbedding(model='text-embedding-ada-002')

### Define Summary Index and Vector Index

In [ ]:
from llama_index.core import SummaryIndex, VectorStoreIndex

summary_index = SummaryIndex(nodes)
vector_index = VectorStoreIndex(nodes)

### Define Query Engines

In [ ]:
summary_query_engine = summary_index.as_query_engine(
    response_mode='tree_summarize',
    use_async=True
)

vector_query_engine = vector_index.as_query_engine()

In [ ]:
from llama_index.core.tools import QueryEngineTool

summary_tool = QueryEngineTool.from_defaults(
    query_engine=summary_query_engine,
    description=(
        'Useful for summarization questions related to MetaGPT'
    )
)

vector_tool = QueryEngineTool.from_defaults(
    query_engine=vector_query_engine,
    description=(
        'Useful for retrieving specific context from MetaGPT'
    )
)

### Define Router Query Engine

In [ ]:
from llama_index.core.query_engine.router_query_engine import RouterQueryEngine
from llama_index.core.selectors import LLMSingleSelector

query_engine = RouterQueryEngine(
    selector=LLMSingleSelector.from_defaults(),
    query_engine_tools=[
        summary_tool,
        vector_tool
    ],
    verbose=True
)

In [ ]:
response = query_engine.query('Summarize the document')

print(str(response))

In [ ]:
len(response.source_nodes)

In [ ]:
response = query_engine.query('How do agents share information with other agents ?')

print(str(response))

In [ ]:
len(response.source_nodes)

### Given a file path create a router query engine

In [ ]:
from llama_index.core import SimpleDirectoryReader, SummaryIndex, VectorStoreIndex
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.query_engine import RouterQueryEngine
from llama_index.core.selectors import LLMSingleSelector
from llama_index.core.tools import QueryEngineTool
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI


def get_router_query_engine(file_path: str, llm = None, embed_model = None):
    """Get router query engine"""
    llm = llm or OpenAI(model='gpt-3.5-turbo')
    embed_model = embed_model or OpenAIEmbedding(model='text-embedding-ada-002')
    
    # Load documents
    documents = SimpleDirectoryReader(input_files=[file_path]).load_data()
    
    splitter = SentenceSplitter(chunk_size=1024)
    nodes = splitter.get_nodes_from_documents(documents)
    
    summary_index = SummaryIndex(nodes)
    vector_index = VectorStoreIndex(nodes)
    
    summary_query_engine = summary_index.as_query_engine(
        response_mode='tree_summarize',
        use_async=True,
        llm=llm
    )
    vector_query_engine = vector_index.as_query_engine()

    summary_tool = QueryEngineTool.from_defaults(
        query_engine=summary_query_engine,
        description=(
            'Useful for summarization questions related to MetaGPT'
        )
    )
    vector_tool = QueryEngineTool.from_defaults(
        query_engine=vector_query_engine,
        description=(
            'Useful for retrieving specific context from MetaGPT'
        )
    )
    
    query_engine = RouterQueryEngine(
        selector=LLMSingleSelector.from_defaults(),
        query_engine_tools=[
            summary_tool,
            vector_tool
        ],
        verbose=True
    )
    
    return query_engine

In [ ]:
query_engine = get_router_query_engine('assets/metagpt.pdf')

In [ ]:
response = query_engine.query('Explain ablation study results')

print(str(response))